<a href="https://colab.research.google.com/github/SpecialAlex/TemporaryStation/blob/main/P03_01_03_Structure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Structure, 구조체
구조체란 여러 데이터를 묶어 새롭게 표현하는 타입으로, 각각의 데이터는 필드(Field)라는 이름으로 구분된다.  
보편적으로 하나의 프로그래밍 언어는 비슷한 기능을 하는 개념으로 구조체와 클래스(Class) 둘 중 하나를 선택하게 되는데, C를 비롯해서 FORTRAN, Matlab, 등의 절차지향적 성격이 강한 언어들은 구조체를 선택하고, Python, Java, R 과 같은 객체지향 언어들은 클래스를 선택하는 경향이 있다.  
  
Julia는 그중 구조체를 선택했다.  
클래스를 설명할 때 객체지향 프로그래밍에서 말하는 캡슐화, 상속, 다향성 등을 항상 언급하지만, 보통 이런 요소들은 Julia를 사용하는 작업에서 크게 중요하지 않을 가능성이 높다.  
프로그래밍에서 단순하고 기능이 적은 것이 꼭 단점은 아닌 게, 별도로 공부할 필요도 없고 코드의 가독성도 좋아지는 데다 대체로는 성능적으로도 뛰어나다.  
상식적으로 생각해봐도 다른 모든 조건이 똑같다면 복잡하게 많은 요소들이 있는 것보단 간단하게 필요한 것만 구현한 쪽이 빠를 수 밖에 없다.
## 필드와 프로퍼티
앞서 구조체가 포함하는 데이터는 필드라는 이름으로 구분된다 하였다.  
간단한 예시로 유리수를 나타내는 타입인 Rational이 구조체로 어떤 필드를 가지는지 확인해 본다.  
`fieldnames()`함수는 주어진 구조체의 필드 이름을 심볼(Symbol)로 반환한다.

In [1]:
half = 2 // 4
println(typeof(half))
println()
println(fieldnames(Rational))
println(getfield(half, :num))
println(getfield(half, :den))

Rational{Int64}

(:num, :den)
1
2


변수 `half`는 유리수 2/2로 정의되었으나 약분을 거친 결과 `1//2`로 표현된다.  
개념적으로 이러한 기약분수는 분자(Numerator)와 분모(Denominator) 두 값으로 구분할 수 있으며, Julia의 유리수라고 하는 구조체 역시 필드로 이 둘을 가진다.  
구체적으로 데이터에서 특정 필드에 접근해서 값을 얻으려면 `getfiled()`함수를 사용한다.  
  
하지만 실전적으로는 비슷한 표현으로 필드보다 프로퍼티(Property)를 훨씬 자주 사용한다.  
굳이 데이터의 타입을 파악해서 그 필드의 이름을 알아낼 필요 없이, `propertynames()`와 `getproperty()`함수로 같은 결과를 얻을 수 있다.

In [2]:
println(propertynames(half))
println(getproperty(half, :num))
println(getproperty(half, :den))

(:num, :den)
1
2


`propertynames(x)`는 근본적으로 `fieldnames(typeof(x))`와 같다.  
실전적으로 사용하는 함수로는 큰 의미 없지만, 이를 통해 알 수 있는 사실은 Julia에서 구조체의 인스턴스(Instance)를 오브젝트(Object)라 부르며 구조체 그 자체가 가지는 속성(Attribute)은 필드(Field)고, 그것의 인스턴스로 실재하는 오브젝트의 속성을 프로퍼티(Property)라 부른다.  
  
한편 `getproperty()`함수는 `data.prop`와 같이 가운데 점을 찍는 점 표기 형태로 쓰여서 데이터 `data`의 프로퍼티 `prop`에 바로 접근할 수 있다.  
예컨대 `getproperty(half, :num)`과 `half.num`은 같은 표현이다.

In [4]:
println(half.num)
println(half.den)

1
2


그렇다면 굳이 적기도 불편한 `getproperty()`함수를 쓸 일이 있을까?  
당연히 있으며, 알아두면 매우 유용하다.  
`getproperty()`함수는수점 표기와는 달리 함수 꼴로 쓰였을 때 브로드캐스트를 사용할 수 있기 떄문에 특정 배열에서 각자의 프로퍼티에 병렬적으로 접근할 수 있다.

In [5]:
Q = [ k // 12 for k in 1:12 ]

12-element Vector{Rational{Int64}}:
  1//12
  1//6
  1//4
  1//3
  5//12
  1//2
  7//12
  2//3
  3//4
  5//6
 11//12
   1

In [6]:
@show getproperty.(Q, :num)

getproperty.(Q, :num) = [1, 1, 1, 1, 5, 1, 7, 2, 3, 5, 11, 1]


12-element Vector{Int64}:
  1
  1
  1
  1
  5
  1
  7
  2
  3
  5
 11
  1

`getproperty()`함수와 브로드캐스트를 사용하면 집합 $Q = \left\{\frac{k}{12}:k = 1, 2, \cdots, 12 \right\}$의 분자만 취할 수 있다.  
새 배열을 만들고 반복문을 돌리는 것보다는 훨씬 간단하다.
## Constructor, 생성자
생성자란 새로운 오브젝트를 만들어주는 함수로 구조체와 정확히 같은 이름으로 정의된다.  
간단한 예로 정수의 값을 길이 `length`로 가지며 끝부분 `tip`에 문자열이 달려있는 구조체를 `Stick`이라는 이름으로 정의해 본다.  
구조체를 정의하는 키워드는 `struct`로, 다음과 같이 타입 정의 `::`를 포함할 수 있다.

In [5]:
# 정석적으로는 "length::Integer" 이름과 데이터 유형 사이에 공백을 두지 않는다.
struct Stick
    length :: Integer
    tip :: String
end

구조체 `Stick`의 오브젝트를 만드는 방법은 간단하다.  `Stick`함수의 필드값에 들어갈 요소들을 순서대로 넣어주는 것이다.  
실제로 프로퍼티들을 참조해보면 우리가 원하는 정의 그대로 오브젝트가 생성된걸 알 수 있다.

In [7]:
arrow = Stick(12, ">")
println(arrow)
println(arrow.length)
println(arrow.tip)

Stick(12, ">")
12
>


## 다형성
프로그래밍 언어에서 다형성(Polymorphism)이란 여러 타입에 대해 작동할 수 있는 능력 및 성질을 일컫는 개념으로 보통은 어떤 요소가 여러가지 타입에 속할 수 있는 것을 말하고, Julia의 경우엔 특히 같은 이름의 함수가 여러 타입에 대해서 기능하는 점을 가리킨다.  
예를 들어 `length()`라는 함수는 굳이 설명하지 않아도 그 이름 그대로 주어진 요소의 길이를 반환하는데, 타입에 따라서 '길이'라는 것의 개념이 달라져도 상식적이거나 의도한 대로의 기능을 수행한다.

In [2]:
println(length("Dynamics"))
println(length([ 0, 1, 3 ]))

8
3


In [3]:
length

length (generic function with 84 methods)

문자열 "Dynamics"는 글자의 수를, 벡터 [0, 1, 3]은 차원의 수를 반환했다.  
`length()`함수는 문자열과 벡터에 대해서 다형성을 가지고 타입에 따라 다르게 동작했다.  
별도의 인수 없이 `length()`함수만 단독으로 호출해 보면 "length (generic function with 88 methods)라는 메시지가 출력되는데, 이는 `length()`함수가 문자열과 벡터를 포함해서 88 가지의 타입에 대해서 각각 '길이'라고 하는 개념에 대응되는 서로 다른 함수로 정의되었음을 말한다.

In [8]:
length(arrow)

LoadError: MethodError: no method matching length(::Stick)
The function `length` exists, but no method is defined for this combination of argument types.

[0mClosest candidates are:
[0m  length([91m::ExponentialBackOff[39m)
[0m[90m   @[39m [90mBase[39m [90m[4merror.jl:271[24m[39m
[0m  length([91m::CompositeException[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mtask.jl:51[24m[39m
[0m  length([91m::Core.MethodTable[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mreflection.jl:1319[24m[39m
[0m  ...


그러나 `arrow`는 길이 12의 `Stick`타입으로 정의되었음에도 `length()`함수가 매칭되는 타입이 없다며 `MethodError`를 일으킨다.  
`Stick`도 개념적으로 길이가 있으니 이 함수에 대응할 수 있도록 한다면 다음과 같이 `Stick`에 맞는 새로운 `length()`함수를 따로 정의해 줘야 한다.

In [9]:
Base.length(x::Stick) = x.length

println(length(arrow))

12


이는 Julia가 코드를 실행시키면서 `length()`라는 함수가 무엇을 위해서 정의되었는지 살펴보고, 정의에서 타입 주석 `x::Stick`을 보고 이것이 `Stick`에 대한 함수라는 힌트를 받은 것이다.  

In [11]:
length.([ "Dynamics", [ 0, 1, 3 ], arrow ])

3-element Vector{Int64}:
  8
  3
 12

다형성을 갖춘다는 것은 이렇게 개념적으로 길이를 원할 때 타입이 달라도 `length()` 단 하나의 함수만 알면 된다는 것이다.  
각자의 구현은 모두 다르겠지만, 사용하는 입장에선 '상식적으로 있을법한'함수만 찾으면 된다.  
  
한편 Julia 초심자가 구조체를 배우면서 다형성의 개념을 반드시 접해야 하는 이유는 `Base.show()`함수를 재정의할 줄 알아야 하기 때문이다.  
`Base.show()`함수는 Julia에서 출력을 담당하는 함수로, 해당 데이터가 호출되었을때 화면에 어떻게 보이는지를 결정한다.

In [12]:
println(arrow)

Stick(12, ">")


In [17]:
function Base.show(io::IO, data::Stick)
    print(io, ("-" ^ data.length) * data.tip)
end

println(arrow)

------------>


원래 `arrow`는 `Stick(12, ">")`라고 하는 정의 그대로의 모습으로 출력되었지만  
`Base.show`함수를 `Stick`에 대해 재정의하여 하이픈(`-`)이 `length`만큼 반복되고 마지막에 `tip`을 붙여서 출력되도록 하면, `arrow`가 화살 같은 모양으로 출력되는 것을 확인할 수 있다.  
  
Julia에서 데이터가 예쁘게 출력되는 것은 꽤 중요한 일로, 앱이나 웹 같은 개발과 거리가 먼 만큼  
콘솔에 출력되는 것 만큼은 보기가 좋아야 하고, 실제 수많은 라이브러리들이 이러한 시각화에 진심이다.  

In [18]:
println(Stick(2, "0"))
println(Stick(4, "I"))
println(Stick(8, "E"))

--0
----I
--------E
